In [1]:
import numpy as np
import pandas as pd
from distortion.geometry import Geometry, bind_metric, local_distortions
np.random.seed(20250621)

n_neighbors = 50
data = pd.read_json("data/mammoth_3d.json")
data.columns = ["x", "y", "z"]

In [2]:
from anndata import AnnData
import scanpy as sc

adata = AnnData(X=data, obs=pd.DataFrame(index=data.index))
sc.pp.neighbors(adata, n_neighbors=n_neighbors)
sc.tl.umap(adata)
embedding = adata.obsm["X_umap"].copy()

In [3]:
radius = 3 * np.mean(adata.obsp["distances"].data)
geom = Geometry("brute", laplacian_method="geometric", affinity_kwds={"radius": radius}, adjacency_kwds={"n_neighbors": n_neighbors}, laplacian_kwds={"scaling_epps": 5})
H, Hvv, Hs = local_distortions(embedding, data, geom)
embedding = bind_metric(embedding, Hvv, Hs)

In [4]:
import plotly.express as px

fig = px.scatter_3d(data, x='x', y='z', z='y')
fig.update_traces(marker_size=1, marker_color="black")
fig

In [5]:
import altair as alt
alt.data_transformers.enable("vegafusion")

alt.Chart(embedding).mark_circle(color="black").encode(
    x=alt.X("embedding_0"),
    y=alt.Y("embedding_1")
).properties(width=450, height=350)

alt.Chart(...)

In [6]:
from distortion.geometry import neighborhood_distances

distances = neighborhood_distances(adata)
alt.Chart(distances).mark_boxplot(outliers=True).encode(
    x=alt.X('true', bin={"maxbins": 20}),
    y=alt.Y('embedding')
)

alt.Chart(...)

In [7]:
from distortion.geometry import neighborhoods
from distortion.visualization import dplot

plots = {}
N = neighborhoods(adata, threshold=.1, outlier_factor=3)
plots["mamoth_links"] = dplot(embedding, height = 350, width=450)\
    .mapping(x="embedding_0", y="embedding_1")\
    .inter_edge_link(N=N, strokeWidth=.2, opacity=0.7, threshold=4, stroke="#F25E7A", highlightColor="#C83F58")\
    .geom_ellipse(opacity=0.8, radiusMin=.5, radiusMax=10)

In [8]:
metrics = {k: H[k] for k in range(len(H))}
plots["mammoth_isometry"] = dplot(embedding, height=350, width=450)\
    .mapping(x="embedding_0", y="embedding_1")\
    .inter_isometry(metrics=metrics, transformation_bw=.5, metric_bw=.1)\
    .geom_ellipse(opacity=0.8, radiusMin=.5, radiusMax=10)

In [9]:
plots["mammoth_box"] = dplot(embedding, height=350, width=530)\
    .mapping(x="embedding_0", y="embedding_1")\
    .geom_ellipse(radiusMax=10, radiusMin=.5)\
    .inter_boxplot(dists=distances, outlier_iqr=10, highlightColor="#F25E7A", strokeWidth=0.4)

In [10]:
[display(p) for p in plots.values()]

dplot(dataset=[{'embedding_0': 8.870267868041992, 'embedding_1': 0.8856074810028076, 'x0': -0.8852808564862759…

dplot(dataset=[{'embedding_0': 8.870267868041992, 'embedding_1': 0.8856074810028076, 'x0': -0.8852808564862759…

dplot(dataset=[{'embedding_0': 8.870267868041992, 'embedding_1': 0.8856074810028076, 'x0': -0.8852808564862759…

[None, None, None]

In [11]:
#[p.save(f"../paper/figures/{k}.svg") for k, p in plots.items()]

In [12]:
len(N)

425